In [80]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp	
import pandas as pd

In [81]:
form2_path = "form2.xlsx"
form1_path = "form1.xlsx"
form2 = pd.read_excel(form2_path, sheet_name='2023年统计的相关数据')
# print(form2.head())
form1 = pd.read_excel(form1_path, sheet_name='乡村种植的农作物')
# 导入农作物信息
crop_num_name = {}
crop_name_num = {}
for index, row in form1.iterrows():
    if index <= 40:
        crop_id = row['作物编号']
        crop_name = row['作物名称']
        crop_num_name[crop_id] = crop_name
        crop_name_num[crop_name] = crop_id
    else: break
# print(crop_num_name)
# print(crop_name_num)

# 导入地块类型
form4 = pd.read_excel(form1_path, sheet_name='乡村的现有耕地')
field_num_type = {}
for index, row in form4.iterrows():
    field_num_type[row['地块名称']] = (row['地块类型'],row['地块面积/亩']) 
print(field_num_type)
y = {} #亩产量
c = {} #种植成本
p = {} #销售单价

for index, row in form2.iterrows():
    if index <= 106:
        crop_id = crop_name_num[row['作物名称']]
        land_type = row['地块类型']
        season = row['种植季次']
    
    # 键使用 (crop_id, land_type, season) 的组合来唯一确定变量
        key = (crop_id, land_type, season)
    
    # 亩产量转换为吨 (1 吨 = 2000 斤)
        y[key] = row['亩产量/斤']
    
    # 种植成本 (元/亩)
        c[key] = row['种植成本/(元/亩)']
    
    # 销售单价范围，取均值
        price_range = row['销售单价/(元/斤)']
        low_price, high_price = map(float, price_range.split('-'))
        p[key] = (low_price + high_price) / 2.0
        if land_type == '智慧大棚' and season == '第二季':
            y[(crop_id,land_type,'第一季')] = row['亩产量/斤']
        else:
            y[(crop_id,land_type,'第二季')] = row['亩产量/斤']
    else: break
    print(y)
    print(p)
    print(c)

{'A1': ('平旱地', 80.0), 'A2': ('平旱地', 55.0), 'A3': ('平旱地', 35.0), 'A4': ('平旱地', 72.0), 'A5': ('平旱地', 68.0), 'A6': ('平旱地', 55.0), 'B1': ('梯田', 60.0), 'B2': ('梯田', 46.0), 'B3': ('梯田', 40.0), 'B4': ('梯田', 28.0), 'B5': ('梯田', 25.0), 'B6': ('梯田', 86.0), 'B7': ('梯田', 55.0), 'B8': ('梯田', 44.0), 'B9': ('梯田', 50.0), 'B10': ('梯田', 25.0), 'B11': ('梯田', 60.0), 'B12': ('梯田', 45.0), 'B13': ('梯田', 35.0), 'B14': ('梯田', 20.0), 'C1': ('山坡地', 15.0), 'C2': ('山坡地', 13.0), 'C3': ('山坡地', 15.0), 'C4': ('山坡地', 18.0), 'C5': ('山坡地', 27.0), 'C6': ('山坡地', 20.0), 'D1': ('水浇地', 15.0), 'D2': ('水浇地', 10.0), 'D3': ('水浇地', 14.0), 'D4': ('水浇地', 6.0), 'D5': ('水浇地', 10.0), 'D6': ('水浇地', 12.0), 'D7': ('水浇地', 22.0), 'D8': ('水浇地', 20.0), 'E1': ('普通大棚 ', 0.6), 'E2': ('普通大棚 ', 0.6), 'E3': ('普通大棚 ', 0.6), 'E4': ('普通大棚 ', 0.6), 'E5': ('普通大棚 ', 0.6), 'E6': ('普通大棚 ', 0.6), 'E7': ('普通大棚 ', 0.6), 'E8': ('普通大棚 ', 0.6), 'E9': ('普通大棚 ', 0.6), 'E10': ('普通大棚 ', 0.6), 'E11': ('普通大棚 ', 0.6), 'E12': ('普通大棚 ', 0.6), 'E13': ('普通大棚 ', 0.6), 'E14'

In [82]:
print(p)

{(1, '平旱地', '单季'): 3.25, (2, '平旱地', '单季'): 7.5, (3, '平旱地', '单季'): 8.25, (4, '平旱地', '单季'): 7.0, (5, '平旱地', '单季'): 6.75, (6, '平旱地', '单季'): 3.5, (7, '平旱地', '单季'): 3.0, (8, '平旱地', '单季'): 6.75, (9, '平旱地', '单季'): 6.0, (10, '平旱地', '单季'): 7.5, (11, '平旱地', '单季'): 40.0, (12, '平旱地', '单季'): 1.5, (13, '平旱地', '单季'): 3.25, (14, '平旱地', '单季'): 5.5, (15, '平旱地', '单季'): 3.5, (1, '梯田', '单季'): 3.25, (2, '梯田', '单季'): 7.5, (3, '梯田', '单季'): 8.25, (4, '梯田', '单季'): 7.0, (5, '梯田', '单季'): 6.75, (6, '梯田', '单季'): 3.5, (7, '梯田', '单季'): 3.0, (8, '梯田', '单季'): 6.75, (9, '梯田', '单季'): 6.0, (10, '梯田', '单季'): 7.5, (11, '梯田', '单季'): 40.0, (12, '梯田', '单季'): 1.5, (13, '梯田', '单季'): 3.25, (14, '梯田', '单季'): 5.5, (15, '梯田', '单季'): 3.5, (1, '山坡地', '单季'): 3.25, (2, '山坡地', '单季'): 7.5, (3, '山坡地', '单季'): 8.25, (4, '山坡地', '单季'): 7.0, (5, '山坡地', '单季'): 6.75, (6, '山坡地', '单季'): 3.5, (7, '山坡地', '单季'): 3.0, (8, '山坡地', '单季'): 6.75, (9, '山坡地', '单季'): 6.0, (10, '山坡地', '单季'): 7.5, (11, '山坡地', '单季'): 40.0, (12, '山坡地', '单季'): 1.5, (13, '山坡地', '单季'

In [83]:
print(c)

{(1, '平旱地', '单季'): 400.0, (2, '平旱地', '单季'): 400.0, (3, '平旱地', '单季'): 350.0, (4, '平旱地', '单季'): 350.0, (5, '平旱地', '单季'): 350.0, (6, '平旱地', '单季'): 450.0, (7, '平旱地', '单季'): 500.0, (8, '平旱地', '单季'): 360.0, (9, '平旱地', '单季'): 400.0, (10, '平旱地', '单季'): 360.0, (11, '平旱地', '单季'): 350.0, (12, '平旱地', '单季'): 1000.0, (13, '平旱地', '单季'): 2000.0, (14, '平旱地', '单季'): 400.0, (15, '平旱地', '单季'): 350.0, (1, '梯田', '单季'): 400.0, (2, '梯田', '单季'): 400.0, (3, '梯田', '单季'): 350.0, (4, '梯田', '单季'): 350.0, (5, '梯田', '单季'): 350.0, (6, '梯田', '单季'): 450.0, (7, '梯田', '单季'): 500.0, (8, '梯田', '单季'): 360.0, (9, '梯田', '单季'): 400.0, (10, '梯田', '单季'): 360.0, (11, '梯田', '单季'): 350.0, (12, '梯田', '单季'): 1000.0, (13, '梯田', '单季'): 2000.0, (14, '梯田', '单季'): 400.0, (15, '梯田', '单季'): 350.0, (1, '山坡地', '单季'): 400.0, (2, '山坡地', '单季'): 400.0, (3, '山坡地', '单季'): 350.0, (4, '山坡地', '单季'): 350.0, (5, '山坡地', '单季'): 350.0, (6, '山坡地', '单季'): 450.0, (7, '山坡地', '单季'): 500.0, (8, '山坡地', '单季'): 360.0, (9, '山坡地', '单季'): 400.0, (10, '山坡地', '单季'): 360.

In [84]:
# 2023年种植情况
form3 = pd.read_excel(form2_path, sheet_name='2023年的农作物种植情况')
print(form3.head())
# 计算预期销售量
expected_sales = [0 for _ in range(len(crop_name_num))]
for index, row in form3.iterrows():
    expected_sales[row['作物编号'] - 1] += row['种植面积/亩'] * y[(row['作物编号'], field_num_type[row['种植地块']][0], row['种植季次'])]
print(expected_sales)

  种植地块  作物编号 作物名称    作物类型  种植面积/亩 种植季次
0   A1     6   小麦      粮食    80.0   单季
1   A2     7   玉米      粮食    55.0   单季
2   A3     7   玉米      粮食    35.0   单季
3   A4     1   黄豆  粮食（豆类）    72.0   单季
4   A5     4   绿豆  粮食（豆类）    68.0   单季
[57000.0, 21850.0, 22400.0, 33040.0, 9875.0, 170840.0, 132750.0, 71400.0, 30000.0, 12500.0, 1500.0, 35100.0, 36000.0, 14000.0, 10000.0, 21000.0, 36240.0, 26880.0, 6240.0, 30000.0, 36210.0, 45360.0, 900.0, 2610.0, 3480.0, 3930.0, 4500.0, 35480.0, 13050.0, 2850.0, 1200.0, 3300.0, 1620.0, 1800.0, 150000.0, 100000.0, 36000.0, 9000.0, 7200.0, 18000.0, 4200.0]


In [85]:
env = gp.Env()
env.setParam('DualReductions', 0)
model_question1_1 = gp.Model("crop_optimization_11",env=env)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Set parameter DualReductions to value 0


In [93]:
# 定义变量
indices = [(crop, field, season)  
        for crop in crop_num_name.keys()  
        for field in field_num_type  
        for season in range(1, 15)]  
x = model_question1_1.addVars(indices, lb=0, vtype=GRB.CONTINUOUS, name="x")
profit = gp.LinExpr()
for item in indices:
    crop = item[0]
    field = item[1]
    season = item[2]
    if (crop,field_num_type[field][0],season) in y.keys() and (crop,field_num_type[field][0],season) in p.keys() and (crop,field_num_type[field][0],season) in c.keys():
        profit += x[item] * p[(crop,field_num_type[field][0],season)] - c[(crop,field_num_type[field][0],season)] * x[item]
    else:
        print(1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [87]:
model_question1_1.setObjective(profit, GRB.MAXIMIZE)

In [88]:
#每块地种植农作物不超过面积
for season in range(1,15):
    for field in field_num_type:
        model_question1_1.addConstr(sum(x[(crop, field, season)] for crop in crop_num_name.keys()) <= field_num_type[field][1])

In [89]:
#农作物种植类型约束
food_crop = []
vegetable_crop = []
fungi_crop = []
bean_crop = [1,2,3,4,5,17,18,19]
for i in range(1,42):
    if i <= 15:
        food_crop.append(i)
    elif i <=37 and i > 16:
        vegetable_crop.append(i)
    elif i >= 38 and i <= 41:
        fungi_crop.append(i)

In [90]:
#平旱地、梯田和山坡地适宜每年种植一季粮食类作物
#水浇地适宜每年种植一季水稻或两季蔬菜，大白菜、白萝卜和红萝卜只能在第二季种植
#普通大棚适宜每年种植一季蔬菜和一季食用菌。普通大棚每年种植两季作物，第一季可种植多种蔬菜（大白菜、白萝卜和红萝卜除外），第二季只能种植食用菌。
#智慧大棚适宜每年种植两季蔬菜，大白菜、白萝卜和红萝卜除外。

for item in indices:
    crop = item[0]
    field = item[1]
    season = item[2]
    if field == '平旱地' or field == '梯田' or field == '山坡地':
        if crop in [vegetable_crop,fungi_crop,16]:
            model_question1_1.addConstr(x[item] == 0)
    elif field == '水浇地':
        if crop in [food_crop,fungi_crop]:
            model_question1_1.addConstr(x[item] == 0)
    elif field == '普通大棚':
        if crop in [food_crop,16]:
            model_question1_1.addConstr(x[item] == 0)
    else:
        if crop in [food_crop,fungi_crop,35,36,37]:
            model_question1_1.addConstr(x[item] == 0)
    
for crop in food_crop:
    for season in range(1,15,2):
        model_question1_1.addConstrs(x[crop,field,season] * x[crop,field,season + 1] == 0 for field in field_num_type if field_num_type[field][0] in ['平旱地', '梯田', '山坡地'])
for season in range(1,15,2):
    model_question1_1.addConstrs(x[16,field, season] * x[16,field, season + 1] == 0 for field in field_num_type if field_num_type[field][0] == '水浇地')

for crop in [fungi_crop,35,36,37]:
    for season in range(1,15,2):
        model_question1_1.addConstrs(x[(crop, field, season)] == 0 for field in field_num_type if field_num_type[field][0] == '普通大棚')
for crop in vegetable_crop:
    for season in range(2,15,2):
        model_question1_1.addConstrs(x[(crop, field, season)] == 0 for field in field_num_type if field_num_type[field][0] == '普通大棚')

for crop in [35,36,37]:
    for season in range(1,15,2):
        model_question1_1.addConstrs(x[(crop, field, season)] == 0 for field in field_num_type if field_num_type[field][0] == '水浇地')



# 白菜和萝卜在第二季每块地只能种植一种
for season in range(2,15,2):
    for crop in vegetable_crop:
        if crop not in [35,36,37]:
            model_question1_1.addConstrs(x[(crop, field, season)] == 0 for field in field_num_type if field_num_type[field][0] == '水浇地')
for season in range(2,15,2):
    for field in field_num_type:
        if field_num_type[field][0] == '水浇地':
            model_question1_1.addConstr(x[(35, field, season)] * x[(36, field, season)] == 0)   
            model_question1_1.addConstr(x[(36, field, season)] * x[(37, field, season)] == 0)


# 任意一块地都不能连续种植
for field in field_num_type:
    for crop in crop_num_name.keys():
        for season in range(1,14):
            model_question1_1.addConstr(x[(crop, field, season)] * x[(crop, field, season + 1)] == 0)

# 因含有豆类作物根菌的土壤有利于其他作物生长，从 2023 年开始要求每个地块（含大棚）的所有土地三年内至少种植一次豆类作物
for crop in bean_crop:
    for season in range(1,9,2):
        model_question1_1.addConstrs(x[(crop, field, season)] + x[(crop, field, season+1)] + x[(crop, field, season+2)] + x[(crop, field, season+3)] + x[(crop, field, season+4)] + x[(crop, field, season+5)] >= field_num_type[field][1] for field in field_num_type)

In [91]:
model_question1_1.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 5928 rows, 30996 columns and 44808 nonzeros
Model fingerprint: 0x8ded8cde
Model has 31680 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 9e+01]
Presolve removed 3444 rows and 3276 columns

Continuous model is non-convex -- solving as a MIP

Presolve added 79326 rows and 51904 columns
Presolve time: 0.18s
Presolved: 85254 rows, 82900 columns, 203340 nonzeros
Variable types: 27720 continuous, 55180 integer (55180 binary)
Root relaxation presolved: 85254 rows, 82900 columns, 203340 nonzeros

Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal 